In [1]:
from pathlib import Path

from src.config import DOCUMENTS_DIR, TEST_DIR
from src.llm_query import call_llm
from src.retriever import jaccard_retriever, get_fittest_chunk, tfidf_retriever
from src.chunk import trunkate_on_h2, find_chunk_indices_with_title
import requests
import json


path = Path(DOCUMENTS_DIR)


2025-02-13 07:16:06.369 | INFO     | src.config:<module>:8 - PROJ_ROOT path is: /home/arys/projects/nlp_rag


In [2]:
texts = []
for filename in path.glob("*.md"):
    with open(filename) as f:
        texts.append(f.read())


## trucate file into chunks

In [3]:
chunks = trunkate_on_h2(texts)

In [4]:
chunks

[{'title': 'burn::tensor# Struct T',
  'chunk': 'burn::tensor# Struct T|||: burn::tensor# Struct T\n`pub struct T;\n`</pre><p>Transpose marker (zero-size type). Used to sugar the transpose of a tensor, e.g.\n<div class="example-wrap">`use burn_tensor::backend::Backend;\nuse burn_tensor::{Tensor, T};\n\nfn example <B: Backend> () {\n    let device = Default::default();\n    let tensor = Tensor:: <B, 2> ::from_floats([[1.0, 2.0], [3.0, 4.0]], &amp;device);\n    let transposed = tensor^T;\n}\n`</pre>\n</div>',
  'index': 0},
 {'title': 'burn::tensor# Struct T',
  'chunk': 'burn::tensor# Struct T|||:  Trait Implementations\n<section class="impl" id="impl-BitXor%3CT%3E-for-Tensor%3CB,+D%3E">Source`impl <B, const D: usize>  BitXor <T>  for Tensor <B, D> <div class="where">where\n    B: Backend,`\n\n<details class="toggle" open="">Source',
  'index': 1},
 {'title': 'burn::tensor# Struct T',
  'chunk': 'burn::tensor# Struct T|||: # type Output = Tensor <B, D> \n\n<div class="docblock">The resu

In [5]:
chunk_tensors_id = find_chunk_indices_with_title(chunks, "burn::tensor# Struct PrintOptions")

In [6]:
tensor_chunk_list = [chunks[i] for i in chunk_tensors_id]

In [7]:
tensor_chunk_list

[{'title': 'burn::tensor# Struct PrintOptions',
  'chunk': 'burn::tensor# Struct PrintOptions|||: burn::tensor# Struct PrintOptions\n`pub struct PrintOptions {\n    pub threshold: usize,\n    pub edge_items: usize,\n    pub precision: Option <usize> ,\n}\n`</pre><p>Options for Tensor pretty printing\n',
  'index': 238},
 {'title': 'burn::tensor# Struct PrintOptions',
  'chunk': 'burn::tensor# Struct PrintOptions|||:  Fields\n§`threshold: usize\n`number of elements to start summarizing tensor\n§`edge_items: usize\n`number of starting elements and ending elements to display\n§`precision: Option <usize> \n`Precision for floating point numbers\n',
  'index': 239},
 {'title': 'burn::tensor# Struct PrintOptions',
  'chunk': 'burn::tensor# Struct PrintOptions|||:  Implementations\n<section class="impl" id="impl-PrintOptions">Source`impl PrintOptions`\n\n<div class="impl-items"><details class="toggle method-toggle" open="">Source',
  'index': 240},
 {'title': 'burn::tensor# Struct PrintOptions

In [13]:
with open(TEST_DIR / 'questions_evaluation.json', 'r') as f:
    eval_json = json.load(f)
eval_json

[{'question': "Comment modifier la valeur d'un tensor pour modifier directement ses poids ?",
  'indices': [3633],
  'reference_response': "Pour exécuter une opération sur un tenseur et modifier sa valeur, vous pouvez utiliser la méthode `inplace`. Cette méthode vous permet de muter un tenseur en utilisant des opérations propriétaires. Notez que cela ne garantit pas la réutilisation du même tampon de données, mais cela peut être utile si vous souhaitez mettre à jour les poids d'un modèle mutable, par exemple."},
 {'question': 'Comment appliquer une opération exponentielle élément par élément sur un tenseur dans `burn` ?',
  'indices': [3634],
  'reference_response': "Pour appliquer une opération exponentielle élément par élément sur un tenseur, utilisez la méthode `exp`. Cela appliquera la fonction exponentielle à chaque élément du tenseur, ce qui est utile pour des transformations non linéaires dans des modèles d'apprentissage automatique."},
 {'question': 'Comment appliquer une opéra

In [14]:
fittest_doc_id = tfidf_retriever(chunks, eval_json[0]['question'])[0]
fittest_doc = chunks[fittest_doc_id]

In [15]:
fittest_doc

{'title': 'burn::tensor# Struct Tensor',
 'chunk': 'burn::tensor# Struct Tensor|||: # pub fn from_primitive(\n    tensor:  <K as TensorKind <B> > ::Primitive,\n) ->  Tensor <B, D, K> \n\nConverts from a primitive tensor into a tensor.\nSource',
 'index': 3578}

In [16]:
response = call_llm(eval_json[0]['question'], fittest_doc,llm="llama2")

In [17]:
response

'Hello! Based on the context information provided, it seems that you are asking how to modify the values of a tensor directly.\n\nIn the `burn` crate, you can use the `from_primitive()` function to convert a primitive tensor into a tensor object. This function takes a `tensor: <K as TensorKind <B> > ::Primitive` as input and returns a `Tensor <B, D, K>` object.\n\nOnce you have a `Tensor` object, you can use the `mutate()` method to modify the values of the tensor. The `mutate()` method takes a closure that defines the operation to be applied to the tensor, and it returns a new `Tensor` object that represents the modified tensor.\n\nHere is an example of how you could use the `mutate()` method to modify the values of a tensor:\n```\nuse burn::tensor;\n\n// Create a tensor with some initial values\nlet tensor = tensor![1, 2, 3];\n\n// Modify the values of the tensor using the mutate() method\nlet mutated_tensor = tensor.mutate(|value| value * 2);\n\n// The modified tensor now has values

In [19]:
from src.evaluation import evaluate_rag

In [20]:
evaluate_rag(eval_json, tfidf_retriever, call_llm, chunks)

IndexError: invalid index to scalar variable.